# Прогноз дохода клиентов

https://www.kaggle.com/competitions/income-prediction-alfa-campus/overview


## Описание задачи

Данное соревнование являлось итоговым проектом в рамках курса [Прикладной Data Science в финтехе](https://alfa-campus.ru/ds-fintech), организованного Альфа Банком.

Задача прогнозирования дохода имеет особенное значение в Банке.
Эта информация помогает точнее и более релевантно подбирать продукты и условия их приобретения, что в свою очередь вносит существенный вклад в прибыль Банка.
Помимо ценности для Банка, оценка дохода является регуляторным требованием ЦБ в части расчета предельно допустимой кредитной нагрузки для клиента.

Бизнес-задача: спрогнозировать доход клиента за определенный месяц.

Техническая задача для специалиста в Data Science: построить модель машинного обучения, которая на основе предложенных характеристик клиента будет предсказывать числовой признак - доход клиента.

## Импорт библиотек

In [ ]:
%%capture
!pip install catboost
!pip install optuna
!pip install category_encoders

In [ ]:
import numpy as np # linear algebra
import pandas as pd
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.metrics import precision_recall_curve, auc
from sklearn.metrics import classification_report
from sklearn.ensemble import StackingRegressor
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from category_encoders import CatBoostEncoder

from scipy import stats

from catboost import Pool
from catboost import CatBoostClassifier
from catboost import CatBoostRegressor

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Input, Embedding, Concatenate, Dense, Reshape, LSTM, SpatialDropout1D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# если работаем в колабе, подключаем гугл диск
from google.colab import drive
drive.mount('/content/drive')

FILE_PATH = '/content/drive/MyDrive/SF_DS/data_income/'

## Функции

In [ ]:
# функция для расчета метрики WMAE
def weighted_mean_absolute_error(y_true, y_pred, weights):

    '''

    Weighted mean absolute error.

    Parameters
    ----------
    y_true: ndarray
        Ground truth
    y_pred: ndarray
        Array of predictions

    Returns
    -------
    rmsle: float
        Weighted mean absolute error

    References
    ----------
    .. [1] https://kaggle-metrics.readthedocs.io/en/latest/_modules/kaggle_metrics/regression.html

    '''

    return (weights * np.abs(y_true - y_pred)).mean()

## Знакомство с данными, базовый анализ и генерация новых признаков.

In [ ]:
train_df = pd.read_csv(
    FILE_PATH+'train.csv',
    sep=";",
    decimal=",",
    encoding="windows-1251"
)
test_df = pd.read_csv(
    FILE_PATH+'test.csv',
    sep=";",
    decimal=",",
    encoding="windows-1251"
)
train_df.shape, test_df.shape

In [ ]:
train_df.head()

Тренировочные данные содержат более 200 000 строк и 233 признака.
 - 'client_id' - id клиента, уникален для каждой записи,
 - 'feature_date' - дата сбора признаков - последний день месяца, за который выполняется прогноз,
 - 'target' - целевая переменная,
 - 'w' - веса для расчета метрики WMAE

Описание всех признаков можно увидеть в файле [description](https://www.kaggle.com/competitions/income-prediction-alfa-campus/data?select=description.xlsx)

In [ ]:
train_df['feature_date'] = pd.to_datetime(train_df['feature_date'])
test_df['feature_date'] = pd.to_datetime(test_df['feature_date'])
train_df['feature_date'].describe()

In [ ]:
test_df['feature_date'].describe()

Данные предоставлены за период с 30-09-2022 по 31-08-2023, прогноз необходимо сделать на дату 30-09-2023

In [ ]:
train_df.describe(include='object')

Преобразуем признаки, полученные из БКИ, в числовые.

In [ ]:
train_df[train_df.filter(like='bki').columns] = train_df[train_df.filter(like='bki').columns].astype(float)
test_df[train_df.filter(like='bki').columns] = test_df[train_df.filter(like='bki').columns].astype(float)

In [ ]:
basic_col = ['client_id', 'feature_date', 'target', 'w']

cat_feat = [col for col in train_df.columns if (col not in basic_col) and (train_df[col].dtype.name=='object')]
#cat_feat += ['year']
num_feat = [col for col in train_df.columns if (col not in basic_col) and (col not in cat_feat)]
len(cat_feat), len(num_feat)

Посмотрим распределение доли пропущенных значений на трейне и тесте.

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))

sns.distplot(train_df[cat_feat+num_feat].isna().mean(axis=0), ax=ax[0])
ax[0].set_title('Распределение доли пропущенных значений по фичам')
ax[0].set_xlabel('Доля NaN-значений')
ax[0].grid(True)

sns.distplot(train_df[cat_feat+num_feat].isna().mean(axis=1), ax=ax[1])
ax[1].set_title('Распределение доли пропущенных значений по клиентам')
ax[1].set_xlabel('Доля NaN-значений')
ax[1].grid(True)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))

sns.distplot(test_df[cat_feat+num_feat].isna().mean(axis=0), ax=ax[0])
ax[0].set_title('Распределение доли пропущенных значений по фичам')
ax[0].set_xlabel('Доля NaN-значений')
ax[0].grid(True)

sns.distplot(test_df[cat_feat+num_feat].isna().mean(axis=1), ax=ax[1])
ax[1].set_title('Распределение доли пропущенных значений по клиентам')
ax[1].set_xlabel('Доля NaN-значений')
ax[1].grid(True)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))

sns.distplot(train_df[cat_feat+num_feat].isna().mean(axis=1), ax=ax[0])
ax[0].set_title('Распределение доли пропущенных значений по клиентам в train')
ax[0].set_xlabel('Доля NaN-значений')
ax[0].grid(True)

sns.distplot(test_df[cat_feat+num_feat].isna().mean(axis=1), ax=ax[1])
ax[1].set_title('Распределение доли пропущенных значений по клиентам в test')
ax[1].set_xlabel('Доля NaN-значений')
ax[1].grid(True)

Видим, что в датасете присутствует много пропусков, как по фичам, так и по клиентам. Создадим признак с долей пропущенных значений.

In [ ]:
train_df['nan_rate'] = train_df[cat_feat+num_feat].isna().mean(axis=1)
test_df['nan_rate'] = test_df[cat_feat+num_feat].isna().mean(axis=1)

Посмотрим внимательней на признаки для должностей.

In [ ]:
train_df[train_df.filter(like='position').columns].describe()

In [ ]:
train_df['main_last_position_ccode'].value_counts().head(10)

In [ ]:
train_df['main_last_position_ccode'].value_counts().head(30).index

Признак должности имеет много уникальных значений, в том числе присутствуют опечатки, различается регистр записей. Должности привели к единому регистру, убрали опечатки, укрупнили и записали в отдельный csv-файл. Присоединяем этот файл к нашему df по client_id.

In [ ]:
train_prof = pd.read_csv(FILE_PATH+'train_professions.csv')
test_prof = pd.read_csv(FILE_PATH+'test_professions.csv')
train_prof.shape, test_prof.shape

In [ ]:
train_df = train_df.merge(train_prof, on='client_id', suffixes=('','_corr'))
test_df = test_df.merge(test_prof, on='client_id', suffixes=('','_corr'))
train_df[train_df.filter(like='position').columns].describe()

In [ ]:
cat_feat += ['main_last_position_ccode_corr', 'short_position',	'short_position_2w']
num_feat += ['nan_rate']

Количество уникальных значений сократилось, но их все равно очень много. Создадим еще признаки должностей, которые будем обрабатывать как текстовые.

In [ ]:
text_feat =  ['main_last_position_ccode',
 'main_pre_last_position_ccode','part_last_position_ccode',
 'part_pre_last_position_ccode', 'main_last_position_ccode_corr']

for feat in text_feat:
    train_df[feat] = train_df[feat].fillna('no_data').astype(str)
    train_df[feat] = train_df[feat].apply(lambda x: x.lower())
    train_df[feat+'_text'] = train_df[feat]
    test_df[feat] = test_df[feat].fillna('no_data').astype(str)
    test_df[feat] = test_df[feat].apply(lambda x: x.lower())
    test_df[feat+'_text'] = test_df[feat]

text_feat =  [feat+'_text' for feat in text_feat]

Выделим из даты признак года и квартала.

In [ ]:
train_df['year'] = train_df['feature_date'].dt.year
test_df['year'] = test_df['feature_date'].dt.year
train_df['quater'] = train_df['feature_date'].dt.quarter.astype(str) + '/' + train_df['feature_date'].dt.year.astype(str)
test_df['quater'] = test_df['feature_date'].dt.quarter.astype(str) + '/' + test_df['feature_date'].dt.year.astype(str)

In [ ]:
cat_feat += ['year', 'quater']

Добавим новые признаки, сгруппировав имеющиеся признаки по категориям трат и по продолжительности сбора данных.

In [ ]:
#группируем транзакции по типу или по продолжительности сбора данных
transactions_amt_invest = [
   'amount_by_category_30d__summarur_amt__sum__cashflowcategory_name__brokerskie_uslugi',
   'avg_by_category__amount__sum__cashflowcategory_name__investitsii',
   'by_category__amount__sum__eoperation_type_name__pokupka_paja'
]

# Находим среднее значение между тремя заданными выражениями и записываем в колонку sum_amt_invest
train_df['sum_amt_invest'] = (train_df[transactions_amt_invest[0]] + \
                             train_df[transactions_amt_invest[1]] + \
                             train_df[transactions_amt_invest[2]]) / 3
test_df['sum_amt_invest'] = (test_df[transactions_amt_invest[0]] + \
                             test_df[transactions_amt_invest[1]] + \
                             test_df[transactions_amt_invest[2]]) / 3

transactions_amt_hotel = [
   'transaction_category_hotels_sum_amt_d15',
   'transaction_category_hotels_sum_amt_m2',
   'avg_3m_hotels'
]

# Находим среднее значение между тремя заданными выражениями и записываем в колонку sum_amt_hotel
train_df['sum_amt_hotel'] = (train_df['transaction_category_hotels_sum_amt_d15']*2 + \
                             train_df['transaction_category_hotels_sum_amt_m2'] / 2 +\
                             train_df['transaction_category_hotels_sum_amt_d15'] / 3) /2
test_df['sum_amt_hotel'] = (test_df['transaction_category_hotels_sum_amt_d15']*2 + \
                            test_df['transaction_category_hotels_sum_amt_m2'] / 2 +\
                            test_df['transaction_category_hotels_sum_amt_d15'] / 3) /2

transactions_amt_supermarket = [
   'transaction_category_supermarket_sum_cnt_d15',
   'transaction_category_supermarket_sum_amt_m3_4'
]

train_df['sum_amt_supermarket'] = train_df['transaction_category_supermarket_sum_amt_m3_4']/4 + \
100*2*train_df['transaction_category_supermarket_sum_cnt_d15']
test_df['sum_amt_supermarket'] = test_df['transaction_category_supermarket_sum_amt_m3_4']/4 + \
100*2*test_df['transaction_category_supermarket_sum_cnt_d15']

transactions_amt_0_5 = [
   'transaction_category_clothers_shoes_sum_amt_d15'
   #'transaction_category_hotels_sum_amt_d15'
]

transactions_amt_1 = [
   'amount_by_category_30d__summarur_amt__sum__cashflowcategory_name__bilety_na_kontserty_i_v_teatry',
   'amount_by_category_30d__summarur_amt__sum__cashflowcategory_name__hosting',
   'amount_by_category_30d__summarur_amt__sum__cashflowcategory_name__spa_sauny_bani',
   'amount_by_category_30d__summarur_amt__sum__cashflowcategory_name__turisticheskie_agenstva',
   'summarur_1m_miscellaneous_stores',
   'summarur_1m_no_cat',
   'avg_by_category__amount__sum__cashflowcategory_name__odezhda_dlja_beremennyh',
   'avg_by_category__amount__sum__cashflowcategory_name__zdorove',
   'amount_by_category_30d__summarur_amt__sum__cashflowcategory_name__tovary_dlja_detej',
   'avg_by_category__amount__sum__cashflowcategory_name__detskie_igrushki'
]

train_df['sum_transactions_amt_0_5_1'] = train_df[transactions_amt_0_5].sum(axis=1)*2 + \
 train_df[transactions_amt_1].sum(axis=1)
test_df['sum_transactions_amt_0_5_1'] = test_df[transactions_amt_0_5].sum(axis=1)*2 + \
 test_df[transactions_amt_1].sum(axis=1)

transactions_amt_3 = [
   'amount_by_category_90d__summarur_amt__sum__cashflowcategory_name__marketplejsy',
   'amount_by_category_90d__summarur_amt__sum__cashflowcategory_name__nalogi',
   'amount_by_category_90d__summarur_amt__sum__cashflowcategory_name__ohota_i_rybalka',
   'amount_by_category_90d__summarur_amt__sum__cashflowcategory_name__prochie_bilety',
]

train_df['sum_transactions_amt_3'] = (train_df[transactions_amt_3]/3).sum(axis=1)
test_df['sum_transactions_amt_3'] = test_df[transactions_amt_3].sum(axis=1)/3

transactions_amt_6 = [
   'avg_6m_building_services',
   'avg_6m_personal_services',
   'amount_by_category_90d__summarur_amt__sum__cashflowcategory_name__prochie_bilety',
   'avg_6m_transportation'
]

train_df['sum_transactions_amt_6'] = train_df[transactions_amt_6].sum(axis=1)/6
test_df['sum_transactions_amt_6'] = test_df[transactions_amt_6].sum(axis=1)/6

transactions_amt_12 = [
   'sum_rare_countries',
   'avg_by_category__amount__sum__cashflowcategory_name__vydacha_nalichnyh_v_bankomate',
   'by_category__amount__sum__eoperation_type_name__vneshnij_perevod_rur'
]

train_df['sum_transactions_amt_12'] = train_df[transactions_amt_12].sum(axis=1)/12
test_df['sum_transactions_amt_12'] = test_df[transactions_amt_12].sum(axis=1)/12

train_df['sum_transactions_amt_all'] = train_df['sum_transactions_amt_0_5_1'] + \
 train_df['sum_transactions_amt_3'] + train_df['sum_transactions_amt_6'] + \
 train_df['sum_transactions_amt_12'] + train_df['sum_amt_supermarket'] + \
 train_df['sum_amt_hotel'] + train_df['sum_amt_invest']

test_df['sum_transactions_amt_all'] = test_df['sum_transactions_amt_0_5_1'] + \
 test_df['sum_transactions_amt_3'] + test_df['sum_transactions_amt_6'] + \
 test_df['sum_transactions_amt_12'] + test_df['sum_amt_supermarket'] + \
 test_df['sum_amt_hotel'] + test_df['sum_amt_invest']

#используя фичу Число транзакций в день считаем минимальные траты за месяц
train_df['min_spending'] = 100*30*train_df['avg_cnt_daily_transactions_90d']
test_df['min_spending'] = 100*30*train_df['avg_cnt_daily_transactions_90d']

train_df['savings'] = (1- train_df['total_rur_amt_cm_avg_div_v2'])*train_df['total_rur_amt_cm_avg_period_days_ago_v2'] / 2
test_df['savings'] = (1- test_df['total_rur_amt_cm_avg_div_v2'])*test_df['total_rur_amt_cm_avg_period_days_ago_v2'] / 2

train_df['money'] = (train_df['cred_dda_rur_amt_3m_avg']/3 + train_df['curr_rur_amt_cm_avg']/12)
test_df['money'] = (test_df['cred_dda_rur_amt_3m_avg']/3 + test_df['curr_rur_amt_cm_avg']/12)

max_limit = [
    'hdb_bki_total_pil_max_limit',
    'hdb_bki_total_cc_max_limit',
    'bki_total_ip_max_limit',
    'hdb_bki_total_max_limit'
]

train_df['max_limit'] = train_df[max_limit].max(axis=1)
test_df['max_limit'] = test_df[max_limit].max(axis=1)

In [ ]:
num_feat += ['sum_amt_invest', 'sum_amt_hotel', 'sum_amt_supermarket',
'sum_transactions_amt_0_5_1', 'sum_transactions_amt_3', 'sum_transactions_amt_6',
'sum_transactions_amt_12', 'sum_transactions_amt_all', 'min_spending', 'savings',
'money', 'max_limit']

Пропущенные значения в категориальных признаках заполним значением 'no_data'.

In [ ]:
for cat_f in cat_feat:
    train_df[cat_f] = train_df[cat_f].fillna('no_data').astype(str)
    train_df.loc[(train_df[cat_f] == 'NaN') |
            (train_df[cat_f] == 'None') |
            (train_df[cat_f] == 'nan') |
            (train_df[cat_f] == ''), cat_f] = 'no_data'

In [ ]:
for cat_f in cat_feat:
    test_df[cat_f] = test_df[cat_f].fillna('no_data').astype(str)
    test_df.loc[(test_df[cat_f] == 'NaN') |
            (test_df[cat_f] == 'None') |
            (test_df[cat_f] == 'nan') |
            (test_df[cat_f] == ''), cat_f] = 'no_data'

## Отбор признаков

Так как данные содержат очень большое количество признаков, перед проведением EDA выполним отбор наиболее значимых признаков.

Отбор будем производить следующим образом:
- по доле пропущенных значений: удалим фичи, у которых доля пропущенных значений равна 1, то есть вообще нет заполненных значений,
- по доле самого частого значения: удалим фичи, которые имеют только одно уникальное значение,
- по рандомной фиче: постром простую модель, добавив рандомный признак и выберем фичи, которые имеют значимость выше рандомной,
- по корреляции: удалим признаки со значением больше 0.95 (из пары коррелирующих фичей удалим менее заполненную фичу).


In [ ]:
# по пропущенным значениям

def feats_filter_nan(df: pd.DataFrame,
                     num_feat: list(),
                     cat_feat: list(),
                     nan_threshold: float):

    """Функция отбора фичей по доле NaN-значений.
    Фичи, которые имеют долю NaN-значений не ниже заданного nan_threshold, подлежат удалению.

    Parameters
    ----------
    df : pd.DataFrame
        датафрейм с фичами
    num_feat : list
        список числовых фичей
    cat_feat : list
        список категориальных фичей (в нашем случае для них NaN-значением является 'no_data')
    nan_threshold : float
        порог удаления фичи

    Returns
    -------
    list
        список фичей, которые останутся в датафрейме

    """

    selected_num_feat = list(
        df[num_feat].isnull().mean()[df.isnull().mean() <= nan_threshold].index
        )
    selected_cat_feat = [
        feat for feat in cat_feat if df[feat].value_counts(normalize=True)[
            df[feat].value_counts(normalize=True).index=='no_data'
            ].max() <= nan_threshold or
            df[feat].value_counts(normalize=True)[
            df[feat].value_counts(normalize=True).index=='no_data'
            ].max() is np.nan
        ]

    selected_features = list(selected_num_feat) + selected_cat_feat
    return selected_features

selected_features = feats_filter_nan(train_df, num_feat, cat_feat, 1)
selected_features_num = [feat for feat in num_feat if feat in selected_features]
selected_features_cat = [feat for feat in cat_feat if feat in selected_features]
len(selected_features)

In [ ]:
# по доле самого частого значения

def feats_filter_nunique(df: pd.DataFrame,
                         features: list,
                         unique_threshold: float):

    """Функция отбора фичей по доле самого частого значения.
    Фичи, которые имеют 1 уникальное значение (помимо NaN), либо для которых
      доля самого частого значения (включая NaN) не ниже unique_threshold, подлежат удалению.

    Parameters
    ----------
    df : pd.DataFrame
        датафрейм с фичами
    features : list
        список фичей
    unique_threshold : float
        порог удаления фичи

    Returns
    -------
    list
        список фичей, которые останутся в датафрейме

    """
    selected_features = [
        feat for feat in features
        if df[feat].value_counts(normalize=True).max() != 1 and
        df[feat].value_counts(normalize=True, dropna=False).max() < unique_threshold
        ]
    return selected_features

selected_features = feats_filter_nunique(train_df, selected_features, 1)
selected_features_num = [feat for feat in num_feat if feat in selected_features]
selected_features_cat = [feat for feat in cat_feat if feat in selected_features]
len(selected_features)

In [ ]:
# отсечение по рандомной фиче

def feats_filter_random_feat(df: pd.DataFrame,
                             features: list,
                             cat_features: list,
                             text_features: list):

    """Функция отбора фичей при помощи отсечения менее важных фичей чем рандомная фича.
    Фичи, которые имеют в простой модели важность ниже рандомной фичи, подлежат удалению.

    Parameters
    ----------
    df : pd.DataFrame
        датафрейм с фичами
    features : list
        список фичей
    cat_features : list
        список категориальных фичей

    Returns
    -------
    list
        список фичей, которые останутся в датафрейме

    """

    df['random'] = np.random.uniform(0, 1, df.shape[0])

    simple_model = CatBoostRegressor(
        random_state = 42,
        verbose=100,
        early_stopping_rounds=50,
        cat_features=cat_features,
        text_features=text_features,
        task_type='GPU'
        )
    simple_model.fit(df[features + ['random']],df['target'])
    features = pd.Series(
        simple_model.feature_importances_,
        index=simple_model.feature_names_
        )
    selected_features = features[features.values > features.loc['random']].index

    return selected_features

selected_features = feats_filter_random_feat(
    train_df,
    selected_features+text_feat,
    selected_features_cat,
    text_feat
    )
selected_features_num = [feat for feat in num_feat if feat in selected_features]
selected_features_cat = [feat for feat in cat_feat if feat in selected_features]
selected_features_text = [feat for feat in text_feat if feat in selected_features]
len(selected_features)

In [ ]:
# по корреляции

def feats_filter_corr(df: pd.DataFrame,
                      features: list,
                      corr_threshold: float):

    """Функция отбора фичей по корреляции между фичами.
    Фичи, которые коррелируют с показателем не менее corr_threshold, подлежат
      удалению (из пары коррелирующих фичей удаляется менее заполненная фича).
      Используется метод df[features].corr() с корреляцией Пирсона.

    Parameters
    ----------
    df : pd.DataFrame
        датафрейм с фичами
    features : list
        список фичей
    corr_threshold : float
        порог удаления фичи

    Returns
    -------
    list
        список фичей, которые останутся в датафрейме

    """
    del_features = []
    matrix_corr = df[features].corr()
    for feature in matrix_corr.index:
        corr_features = matrix_corr.loc[feature][matrix_corr.loc[feature] >= corr_threshold].index
        for corr_feature in corr_features:
            if feature != corr_feature and df[feature].isnull().sum() < df[corr_feature].isnull().sum():
                del_features.append(corr_feature)
            elif feature != corr_feature and df[feature].isnull().sum() > df[corr_feature].isnull().sum():
                del_features.append(feature)
            elif feature != corr_feature and df[feature].isnull().sum() == df[corr_feature].isnull().sum():
                del_features.append(min(feature, corr_feature))

    del_features = list(set(del_features))
    selected_features = [feat for feat in features if feat not in del_features]
    return selected_features

selected_features_num = feats_filter_corr(train_df, selected_features_num, 0.95)
selected_features = selected_features_num + selected_features_cat + selected_features_text
len(selected_features)

In [ ]:
selected_features, selected_features_cat, selected_features_text

In [ ]:
selected_features = ['accum_rur_amt_cm_avg_div_v2',
  'atravel',
  'avg_3m_hotels',
  'avg_6m_building_services',
  'avg_6m_money_transactions',
  'avg_6m_personal_services',
  'avg_6m_transportation',
  'avg_by_category__amount__sum__cashflowcategory_name__detskie_igrushki',
  'avg_by_category__amount__sum__cashflowcategory_name__zdorove',
  'avg_debet_turn_rur',
  'avg_percents_inc',
  'by_category__amount__sum__eoperation_type_name__vneshnij_perevod_rur',
  'cc_other_rate_max_2avg_prop',
  'channel_mobilnoe_prilozhenie_am_voc_features_12m_mark_eq_1_flag',
  'channel_mobilnoe_prilozhenie_am_voc_features_12m_voc_with_expert_cnt',
  'commission_outcome_rur_amt',
  'curr_rur_amt_cm_avg',
  'hdb_bki_active_cc_cnt',
  'hdb_bki_active_ip_max_outstand',
  'hdb_bki_active_pil_max_overdue',
  'hdb_bki_other_active_auto_month_payments_sum',
  'hdb_bki_total_cc_max_limit',
  'hdb_bki_total_max_overdue_sum',
  'hdb_bki_total_pil_max_limit',
  'hdb_outstand_sum',
  'incomeValue',
  'max_cc_largest_max_limit_actoff_30d',
  'max_pil_largest_max_limit_actoff_90d',
  'min_cc_max_el_actoff_90d',
  'min_max_limit',
  'min_pil_max_score_actoff_180d',
  'min_pil_max_ul_actoff_90d',
  'mob_cover_days',
  'percent_outcome_rur_amt',
  'product_zarplatnaja_karta_voc_features_36m_mark_eq_5_flag',
  'product_zarplatnaja_karta_voc_features_3m_voc_with_expert_cnt',
  'prof_cc_prof',
  'profit_income_out_rur_amt_9m',
  'smsInWavg6m',
  'staff_flag',
  'first_salary_income',
  'summarur_1m_no_cat',
  'total_rur_amt_cm_avg_div_v2',
  'total_rur_amt_cm_avg_period_days_ago_v2',
  'transaction_category_hotels_sum_amt_m2',
  'transaction_category_supermarket_sum_amt_m3_4',
  'transaction_category_supermarket_sum_cnt_d15',
  'turn_cc_cr_max_v2',
  'turn_other_cr_avg_v2',
  'turn_other_db_max_v2',
  'uniV5',
  'worksalary_rur_amt',
  'nan_rate',
  'sum_transactions_amt_0_5_1',
  'sum_transactions_amt_6',
  'sum_transactions_amt_12',
  'min_spending',
  'savings',
  'money',
  'max_limit',
  'addrref',
  'oldest_campaignsegment_ccode_for_nss',
  'oldest_campaignsegment_ccode_for_pil',
  'segment',
  'short_position_2w',
  'year',
  'quater',
  'main_last_position_ccode_text',
  'main_pre_last_position_ccode_text',
  'part_last_position_ccode_text',
  'main_last_position_ccode_corr_text']
selected_features_cat = ['addrref',
  'oldest_campaignsegment_ccode_for_nss',
  'oldest_campaignsegment_ccode_for_pil',
  'segment',
  'short_position_2w',
  'year',
  'quater']
selected_features_text = ['main_last_position_ccode_text',
  'main_pre_last_position_ccode_text',
  'part_last_position_ccode_text',
  'main_last_position_ccode_corr_text']
selected_features_num = [feat for feat in selected_features
                         if feat not in selected_features_cat
                        and feat not in selected_features_text]
len(selected_features), len(selected_features_cat), \
len(selected_features_num), len(selected_features_text)

## EDA

Посмотрим на описательные статистики и распределение целевой переменной.

In [ ]:
train_df['target'].describe()

In [ ]:
fig = px.histogram(train_df, x = 'target', marginal = 'box',
                   title ='Распределение дохода')
fig.show()

Судя по гистограмме, целевая переменная имеет логнормальное распределение с длинным хвостом справа. Возможно, при предсказании будет полезно логорифировать таргет.

Проверим, является ли распределения логарифмированного таргета нормальным.

In [ ]:
# задаём уровень значимости
alpha = 0.05

# проводим тест
_, p = stats.normaltest(np.log(train_df['target']))

print('p-value = ',p)

# интерпретируем результат
if p <= alpha:
    print('Распределение не нормальное')
else:
    print('Распределение нормальное')

Посмотрим визуализацию, которая позволяет сравнить распределение дохода в зависимости от года и квартала.

In [ ]:
fig = px.histogram(train_df, x = 'target', marginal = 'box',
                   color='year',
                   title ='Распределение дохода в разрезе года')
fig.show()

Распределение дохода отличается в зависимости от года, в 2023 году медиана меньше.

Посмотрим на медианный доход клиентов в зависимости от квартала.

In [ ]:
data = train_df.groupby('quater')['target'].median()
fig = px.bar(data, x=data.index, y=data.values,
                   title='Медианный доход в зависимости от квартала',
                   labels={'y':'median of target'})
fig.show()

Видим различия в уровне медианного дохода в зависимости от квартала, то есть временной признак (квартал+год), вероятно, имеет высокую важность при предсказании уровня дохода.

Посмотрим на распределение некоторых признаков и их корреляцию с уровнем дохода.

In [ ]:
fig = px.box(train_df, x='worksalary_rur_amt',
                   title='Распределение зарплаты')
fig.show()

Видим, что признак 'worksalary_rur_amt' содержит анамольно большие значения. Удалим значения выше 99 персентиля и посмотрим на распределение.

In [ ]:
train_df.loc[train_df['worksalary_rur_amt'] > np.percentile(train_df['worksalary_rur_amt'].dropna(),99),
             'worksalary_rur_amt'] = np.nan
test_df.loc[test_df['worksalary_rur_amt'] > np.percentile(train_df['worksalary_rur_amt'].dropna(),99),
            'worksalary_rur_amt'] = np.nan

In [ ]:
fig = px.histogram(train_df, x='worksalary_rur_amt', marginal='box',
                   title='Распределение зарплаты')
fig.show()

In [ ]:
fig = px.histogram(test_df, x='worksalary_rur_amt', marginal='box',
                   title='Распределение зарплаты')
fig.show()

In [ ]:
fig = px.scatter(train_df, x='worksalary_rur_amt', y='target',
                   title='Зависимость между значением зарплаты и доходом')
fig.show()

Наблюдается корреляция между уровнем дохода и уровнем зарплаты.

Посмотрим, какие еще признаки имеют высокий уровень корреляции с таргетом.

In [ ]:
matrix_corr = train_df.corr()

In [ ]:
corr_features = matrix_corr['target'].drop(['target','w']).sort_values(ascending=False).head(20).index

In [ ]:
train_df[corr_features].describe()

In [ ]:
test_df[corr_features].describe()

Признак 'first_salary_income' имеет высокую корреляцию с таргетом на тренировочной выборке, но при этом в тестовой этот признак не заполнен. Вероятно, это лик, который следует исключить из выборки.

In [ ]:
selected_features.remove('first_salary_income')
selected_features_num.remove('first_salary_income')

Посмотрим на распределение дебетовых оборотов и их взаимосвязь с доходом.

In [ ]:
fig = px.histogram(train_df, x='avg_debet_turn_rur', marginal='box',
                   title='Распределение дебетовых оборотов')
fig.show()

Удалим значения выше 99 персентиля.

In [ ]:
train_df.loc[train_df['avg_debet_turn_rur'] > np.percentile(train_df['avg_debet_turn_rur'].dropna(),99),
             'avg_debet_turn_rur'] = np.nan
test_df.loc[test_df['avg_debet_turn_rur'] > np.percentile(train_df['avg_debet_turn_rur'].dropna(),99),
            'avg_debet_turn_rur'] = np.nan

In [ ]:
fig = px.histogram(train_df, x='avg_debet_turn_rur', marginal='box',
                   title='Распределение дебетовых оборотов')
fig.show()

In [ ]:
fig = px.scatter(train_df, x='avg_debet_turn_rur', y='target',
                   title='Зависимость между дебетовыми оборотами и доходом')
fig.show()

Посмотрим есть ли взаимосвязь доли пустых значений и таргета.

In [ ]:
fig = px.scatter(train_df, x='nan_rate', y='target')
fig.show()

Видно, что наибольшие значения таргета присутсвуют для строк с количеством пропусков 60-70% и при этом для доли пропусков > 99% значения таргета также имеют большой разброс. Так как предсказать для этих клиентов доход не представляется возможным для минимизации метрики соревнования WMAE нужно использовать медиану. Посмотрим на медиану по таким строкам, и по всему остальному трейну.

In [ ]:
train_df[train_df['nan_rate'] > 0.995]['target'].median(), \
train_df[train_df['nan_rate'] <= 0.995]['target'].median()

Видим, что медиана по строкам с большим количеством пропусков значительно меньше медианы по остальному датасету.

## Baseline

В качестве бейзлайна будем предсказывать уровень дохода медианой. Для валидации выделим последний месяц из тренировочной выборки.

In [ ]:
tr_data = train_df[train_df['feature_date'] < '30/08/2023']
val_data = train_df[train_df['feature_date'] >= '30/08/2023']
tr_data.shape, val_data.shape

In [ ]:
val_data['preds'] = tr_data['target'].median()

weighted_mean_absolute_error(val_data['target'], val_data['preds'], val_data['w'])

## Модель Catboost

Для построения модели регрессора воспользуемся реализацией градиентного бустинга из библиотеки Catboost. Эта библиотека удобна тем, что не требует заполнения пропусков и предобработки категориальных фичей, а также имеет возможность обработки текстовых фичей.

### Функция потерь RMSE

In [ ]:
train_data = Pool(
    tr_data[selected_features], tr_data['target'],
    weight=tr_data['w'],
    cat_features=selected_features_cat,
    text_features=selected_features_text
)

eval_data = Pool(
    val_data[selected_features], val_data['target'],
    weight=val_data['w'],
    cat_features=selected_features_cat,
    text_features=selected_features_text
)

In [ ]:
model_mse = CatBoostRegressor(
    learning_rate = 0.1,
    iterations=10000,
    random_state = 42,
    verbose=500,
    early_stopping_rounds=200,
    cat_features=selected_features_cat,
    text_features=selected_features_text,
    task_type='GPU',
    border_count=254,
    eval_metric='MAE'
)

In [ ]:
model_mse.fit(train_data,
           eval_set=eval_data)

In [ ]:
y_train_pred_mse = model_mse.predict(tr_data[selected_features])
y_val_pred_mse = model_mse.predict(val_data[selected_features])

weighted_mean_absolute_error(tr_data['target'], y_train_pred_mse, tr_data['w']), \
weighted_mean_absolute_error(val_data['target'], y_val_pred_mse, val_data['w'])

Посмотрим, какие значения предсказывает модель для слабо заполненных строк.

In [ ]:
y_val_pred_mse[val_data['nan_rate'] > 0.995]

Попробуем проставить медиану для таких значений

In [ ]:
val_data['preds'] = y_val_pred_mse
val_data.loc[val_data['nan_rate'] > 0.995, 'preds'] = tr_data[tr_data['nan_rate']>0.995]['target'].median()
weighted_mean_absolute_error(val_data['target'], val_data['preds'], val_data['w'])

### Функция потерь RMSE и логарифмированный таргет

Так как наше распределение похоже на логнормальное, попробуем построить модель для предсказания логарифмированного таргета.

In [ ]:
train_data = Pool(
    tr_data[selected_features], np.log(tr_data['target']),
    weight=tr_data['w'],
    cat_features=selected_features_cat,
    text_features=selected_features_text
)

eval_data = Pool(
    val_data[selected_features], np.log(val_data['target']),
    weight=val_data['w'],
    cat_features=selected_features_cat,
    text_features=selected_features_text
)

In [ ]:
model_log = CatBoostRegressor(
    loss_function = 'RMSE',
    iterations = 10000,
    random_state = 42,
    verbose=500,
    early_stopping_rounds=200,
    cat_features=selected_features_cat,
    text_features=selected_features_text,
    task_type='GPU',
    border_count=254,
    eval_metric='MAE'
)
model_log.fit(train_data,
           eval_set=eval_data)

In [ ]:
y_train_pred_log = np.exp(model_log.predict(tr_data[selected_features]))
y_val_pred_log = np.exp(model_log.predict(val_data[selected_features]))

weighted_mean_absolute_error(tr_data['target'], y_train_pred_log, tr_data['w']), \
weighted_mean_absolute_error(val_data['target'], y_val_pred_log, val_data['w'])

In [ ]:
y_val_pred_log[val_data['nan_rate'] > 0.995]

Модель с логарифмированным таргетом проставила значение близкое к медианным, но вероятно, хуже уловила большие значения.

### Функция потерь MAE

In [ ]:
train_data = Pool(
    tr_data[selected_features], tr_data['target'],
    weight=tr_data['w'],
    cat_features=selected_features_cat,
    text_features=selected_features_text
)

eval_data = Pool(
    val_data[selected_features], val_data['target'],
    weight=val_data['w'],
    cat_features=selected_features_cat,
    text_features=selected_features_text
)

Опытным путем было обнаружено, что при увеличении learning_rate до аномально больших значений модель c лоссом MAE на GPU начинает лучше сходиться.

In [ ]:
model_mae = CatBoostRegressor(
    loss_function = 'MAE',
    iterations = 10000,
    learning_rate=4500, # увеличиваем learning_rate
    random_state = 42,
    verbose=500,
    early_stopping_rounds=200,
    cat_features=selected_features_cat,
    text_features=selected_features_text,
    task_type='GPU',
    border_count=254,
    eval_metric='MAE'
)
model_mae.fit(train_data,
           eval_set=eval_data
             )

In [ ]:
y_train_pred_mae = model_mae.predict(tr_data[selected_features])
y_val_pred_mae = model_mae.predict(val_data[selected_features])

weighted_mean_absolute_error(tr_data['target'], y_train_pred_mae, tr_data['w']), \
weighted_mean_absolute_error(val_data['target'], y_val_pred_mae, val_data['w'])

In [ ]:
y_val_pred_mae[val_data['nan_rate'] > 0.995]

### Важность признаков

Посмотрим на важность признаков для построенных моделей

In [ ]:
# посмотрим на важность признаков
feat_imp = pd.DataFrame({
  'feature': model_mse.feature_names_,
  'importance':model_mse.feature_importances_
  })

print(feat_imp[['feature', 'importance']].sort_values('importance', ascending = False).head(20))

In [ ]:
# посмотрим на важность признаков
feat_imp = pd.DataFrame({
  'feature': model_mae.feature_names_,
  'importance':model_mae.feature_importances_
  })

print(feat_imp[['feature', 'importance']].sort_values('importance', ascending = False).head(20))

### Подбор гиперпараметров

In [ ]:
import optuna

def objective(trial):

    # 1. разделяем случайным образом выборку X_cv, y_cv на обучающую, валидационную и тестовую
    x_train, x_val, y_tr, y_v, w_tr, w_v = train_test_split(
        tr_data[selected_features], tr_data['target'], tr_data['w'], test_size=0.2
        )

    # 2. назначаем параметры и значения для перебора
    param = {
        "depth": trial.suggest_categorical("depth", [6, 10]),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 3, 25),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0, 10.0),
        "random_strength": trial.suggest_float("random_strength", 1, 10),
    }

    train_data = Pool(
        x_train, y_tr,
        weight=w_tr,
        cat_features=selected_features_cat,
        text_features=selected_features_text
    )

    eval_data = Pool(
        x_val, y_v,
        weight=w_v,
        cat_features=selected_features_cat,
        text_features=selected_features_text
    )
    # 3. обучаем модель с выбранными параметрами
    model_cv = CatBoostRegressor(
        **param,
        iterations = 1000,
        random_state = 42,
        verbose=500,
        early_stopping_rounds=200,
        cat_features=selected_features_cat,
        text_features=selected_features_text,
        task_type='GPU',
        border_count=254,
        eval_metric='MAE'
        )
    model_cv.fit(train_data, eval_set=eval_data, verbose=False)

    # 4. делаем предсказание и вовращаем значение метрики
    preds = model_cv.predict(te_data[selected_features])
    metric = weighted_mean_absolute_error(te_data['target'], preds, te_data['w'])

    return metric

In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, timeout=1800)

In [ ]:
print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
best_trial = study.best_trial

print("  Value: {}".format(best_trial.value))

print("  Params: ")
for key, value in best_trial.params.items():
    print("    {}: {}".format(key, value))

In [ ]:
best_trial.params

In [ ]:
best_params = {'depth': 10,
 'l2_leaf_reg': 23.796616602977505,
 'bagging_temperature': 0.3985497275111982,
 'random_strength': 8.179601506035654}

In [ ]:
model_best = CatBoostRegressor(
    **best_params,
    iterations = 10000,
    random_state = 42,
    verbose=500,
    early_stopping_rounds=200,
    cat_features=selected_features_cat,
    text_features=selected_features_text,
    task_type='GPU',
    border_count=254,
    eval_metric='MAE'
)
model_best.fit(train_data,
           eval_set=eval_data)

In [ ]:
y_train_pred = model_best.predict(tr_data[selected_features])
y_val_pred = model_best.predict(val_data[selected_features])

weighted_mean_absolute_error(tr_data['target'], y_train_pred, tr_data['w']), \
weighted_mean_absolute_error(val_data['target'], y_val_pred, val_data['w'])

In [ ]:
val_data['preds'] = y_val_pred
val_data.loc[val_data['nan_rate'] > 0.995, 'preds'] = tr_data[tr_data['nan_rate']>0.995]['target'].median()
weighted_mean_absolute_error(val_data['target'], val_data['preds'], val_data['w'])

Оптимизация гиперпараметров помогла немного улучшить качество предсказания.

## Классификатор

Так как функция потерь MSE лучше предсказывает большие значения, так как сильнее штрафует большие ошибки, а функция MAE лучше предсказывает медианные значения, возникла идея совместить предсказания этих моделей, построив классификатор для классификации клиентов по уровню дохода. Разделим клиентов на 2 группы, добавив признак 'class_target', который равен 1 для дохода выше или равного 200000 и 0 для дохода меньше 200000.

In [ ]:
train_df['class_target'] = train_df['target'].apply(lambda x: 1 if x >= 200000 else 0)

In [ ]:
train_df['class_target'].value_counts()

In [ ]:
train_df.groupby('class_target')['target'].describe()

Выделим валидационную выборку таким же образом, как ранее - по последнему месяцу в тренировочной выборке.

In [ ]:
tr_data = train_df[train_df['feature_date'] < '30/08/2023']
val_data = train_df[train_df['feature_date'] >= '30/08/2023']
tr_data.shape, val_data.shape

In [ ]:
tr_data['class_target'].mean(), val_data['class_target'].mean()

Предсказывать класс будем с помощью CatBoostClassifier, используя кросс-валидацию на 5 фолдов.

In [ ]:
# используем StratifiedKFold, так как классы не сбалансированы
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
pr_auc_kf = []
models_cl = []

for j, (train_index, test_index) in enumerate(kf.split(
    tr_data[selected_features], tr_data['class_target']
    )):
    model = CatBoostClassifier(
        iterations=10000,
        random_state = 42,
        verbose=500,
        early_stopping_rounds=200,
        cat_features=selected_features_cat,
        text_features=selected_features_text,
        task_type='GPU',
        auto_class_weights='Balanced' # для балансировки классов
    )

    train_data = Pool(
        tr_data[selected_features].iloc[train_index],
        tr_data['class_target'].iloc[train_index],
        cat_features=selected_features_cat,
        text_features = selected_features_text
    )

    eval_data = Pool(
        tr_data[selected_features].iloc[test_index],
        tr_data['class_target'].iloc[test_index],
        cat_features=selected_features_cat,
        text_features = selected_features_text,
    )

    model.fit(train_data, eval_set=eval_data)

    preds_test = model.predict_proba(tr_data[selected_features].iloc[test_index])[:, 1]

    precision, recall, thresholds = precision_recall_curve(
        tr_data.iloc[test_index]['class_target'], preds_test)
    pr_auc_kf.append(auc(recall, precision))
    models_cl.append(model)

pr_auc_kf


In [ ]:
# предсказание на кросс-валидации
preds_val_proba = np.zeros([val_data.shape[0], len(models_cl[0].classes_)])

for model_cl in models_cl:
    preds_val_proba += model_cl.predict_proba(val_data[selected_features])/len(models_cl)

preds_val_cl = np.argmax(preds_val_proba, axis=1)

print(classification_report(val_data['class_target'],preds_val_cl))

In [ ]:
precision, recall, thresholds = precision_recall_curve(
    val_data['class_target'], preds_val_proba[:, 1]
)
auc(recall, precision)

Получим итоговое предсказание, используя предсказание модели с MSE для высоких доходов и модели с метрикой MAE для низких зарплат.

In [ ]:
val_data['pred_proba'] = preds_val_proba[:, 1]
val_data['pred_mse'] = y_val_pred
val_data['pred_mae'] = y_val_pred_mae

In [ ]:
val_data['preds'] = val_data['pred_mse']*val_data['pred_proba'] + val_data['pred_mae']*(1-val_data['pred_proba'])

weighted_mean_absolute_error(val_data['target'], val_data['preds'], val_data['w'])

## Stacking

### Stacking - 2 базовые модели

In [ ]:
params_cat = {
              'verbose': 1000,
              'n_estimators': 8000,
              'cat_features' : selected_features_cat,
              'text_features' : selected_features_text,
              'early_stopping_rounds': 200,
              'task_type': 'GPU'
         }

In [ ]:
model_rmse = CatBoostRegressor(**params_cat)
model_mae = CatBoostRegressor(**params_cat, learning_rate=4500, loss_function='MAE')

In [ ]:
# список базовых моделей
estimators = [
    ("CatBoost_rmse", model_rmse),
    ("CatBoost_mae", model_rmse)
]

# в качестве мета-модели будем использовать LGBM
meta_model = StackingRegressor(
    estimators=estimators,
    final_estimator=LGBMRegressor(objective='regression_l1', n_estimators = 1000),
    verbose=3,
)

stacking_reg_2 = meta_model
stacking_reg_2

In [ ]:
stacking_reg_2.fit(tr_data[selected_features], tr_data['target'], sample_weight=tr_data['w'])

In [ ]:
y_pred_val_st_2 = stacking_reg_2.predict(val_data[selected_features])

weighted_mean_absolute_error(val_data['target'], y_pred_val_st_2, val_data['w'])

In [ ]:
for model, (name, _) in zip(stacking_reg_2.estimators_, stacking_reg_2.estimators):
    print(name, 'wmae: ',
          round(weighted_mean_absolute_error(
              model.predict(val_data[selected_features]), val_data['target'], val_data['w']),
              4))

С помощью стекинга мы улучшили предсказания базовых моделей, но качество стекинга уступает качеству предсказания по вероятностям классификатора, построенного ранее.

### Stacking - 5 базовых моделей

Построим еще один вариант стекинга, где в качестве базовых моделей будем использовать 6 базовых моделей, в том числе XGBoost и LGBM.

Предварительно преобразуем категориальные фичи в числовой формат, чтобы можно было подать на вход всем базовым моделям, с помощью CatBoostEncoder.

In [ ]:
catboost_encoder = CatBoostEncoder(cols=selected_features_cat, return_df=True, sigma=0.05)
catboost_encoder.fit(tr_data[selected_features], tr_data['target'])

tr_data[selected_features] = catboost_encoder.transform(tr_data[selected_features])
val_data[selected_features] = catboost_encoder.transform(val_data[selected_features])

In [ ]:
params_xgb = {
    "eta": 0.05,
    "max_depth": 6,
    "subsample": 0.7,
    'min_child_weight' : 0.1,
    'gamma': .01,
    'reg_lambda' : 0.1,
    'reg_alpha' : 0.5,
    "verbosity": 2,
    'tree_method' : 'gpu_hist'

}

In [ ]:
params_cat = {
              'verbose': 1000,
              'n_estimators': 8000,
              'early_stopping_rounds': 200,
              'task_type': 'GPU'
         }

In [ ]:
model_rmse = CatBoostRegressor(**params_cat)
model_lgbm_mse = LGBMRegressor(n_estimators=500)
model_lgbm_mae = LGBMRegressor(objective='regression_l1', n_estimators = 1000)
xgb_model_rmse = XGBRegressor(**params_xgb, n_estimators=1500)
xgb_model_mae = XGBRegressor(**params_xgb, n_estimators=1000, objective='reg:absoluteerror')

In [ ]:
# список базовых моделей
estimators = [
    ("XGB_rmse", xgb_model_rmse),
    ("XGB_mae", xgb_model_mae),
    ("CatBoost_rmse", model_rmse),
    ("LGBM_mse", model_lgbm_mse),
    ("LGBM_mae", model_lgbm_mae),
]

# в качестве мета-модели будем использовать XGBM
meta_model = StackingRegressor(
    estimators=estimators,
    final_estimator=XGBRegressor(**params_xgb, n_estimators=1000, objective='reg:absoluteerror'),
    passthrough=True, # добавим исходные признаки к метапризнакам
    verbose=3,
)

stacking_reg_5 = meta_model
stacking_reg_5


In [ ]:
stacking_reg_5.fit(tr_data[selected_features_num + selected_features_cat],
                 tr_data['target'], sample_weight=train_df['w'])

In [ ]:
y_pred_val_st_6 = stacking_reg_5.predict(val_data[selected_features])

weighted_mean_absolute_error(val_data['target'], y_pred_val_st_5, val_data['w'])

In [ ]:
for model, (name, _) in zip(stacking_reg_6.estimators_, stacking_reg_6.estimators):
    print(name, 'wmae: ',
          round(weighted_mean_absolute_error(
              model.predict(val_data[selected_features], val_data['target'], val_data['w']),
              4)))

Такой стекинг значительно улучшает качество отдельных базовых моделей, и качество предсказания примерно совпадает с качеством предсказния по классификатору.

## NeuralNetwork

Построим нейросеть для решения нашей задачи с помощью фреймворка TensorFlow. Добавим Embedding слои для категориальных и текстовых признаков.

Перед построением нейросети предобработаем данные: переведем в числовой формат категориальные признаки, токенизируем текстовые признаки, а также нормализуем числовые признаки.

In [ ]:
# скопируем тренировочный датасет для выполнения преобразований с ним
train_nn = train_df.copy()
test_nn = test_df.copy()

# переведем признак года в числовой (0 - 2023 год, 1 - 2022 год)
train_nn['year'] = 2023 - train_nn['year'].astype(int)
test_nn['year'] = 2023 - test_nn['year'].astype(int)
selected_features_num += ['year']

In [ ]:
for feat in selected_features_num:  # Для каждого числового признака
    upper_bound = np.percentile(train_nn[feat].dropna(), 99)  # Верхняя граница
    lower_bound = np.percentile(train_nn[feat].dropna(), 1)  # Нижняя граница

    # Заменяем значения, выходящие за границы 99 персентиля
    train_nn.loc[train_nn[feat] > upper_bound, feat] = upper_bound
    train_nn.loc[train_nn[feat] < lower_bound, feat] = lower_bound
    test_nn.loc[test_nn[feat] > upper_bound, feat] = upper_bound
    test_nn.loc[test_nn[feat] < lower_bound, feat] = lower_bound

# заполним пропуски по части признаков 1, по остальным нулем
train_nn.fillna(
    {'accum_rur_amt_cm_avg_div_v2': 1,
     'cc_other_rate_max_2avg_prop': 1,
     'total_rur_amt_cm_avg_div_v2': 1},
    inplace=True
    )
train_nn.fillna(0, inplace=True)

test_nn.fillna(
    {'accum_rur_amt_cm_avg_div_v2': 1,
     'cc_other_rate_max_2avg_prop': 1,
     'total_rur_amt_cm_avg_div_v2': 1},
    inplace=True
    )
test_nn.fillna(0, inplace=True)

In [ ]:
# категориальные признаки закодируем с помощью LabelEncoder
for feat in selected_features_cat:
    le = LabelEncoder()
    le.fit(pd.concat([train_nn[feat], test_nn[feat]]))
    test_nn[feat] = le.transform(test_nn[feat])
    train_nn[feat] = le.transform(train_nn[feat])

In [ ]:
# выделим валидационную выборку последний месяц отдельно по типам фичей
X_train = train_nn[train_nn['feature_date'] <= '30/08/2023'][selected_features_num]
X_val = train_nn[train_nn['feature_date'] > '30/08/2023'][selected_features_num]
X_train_cat = train_nn[train_nn['feature_date'] <= '30/08/2023'][selected_features_cat]
X_val_cat = train_nn[train_nn['feature_date'] > '30/08/2023'][selected_features_cat]
X_train_text = train_nn[train_nn['feature_date'] <= '30/08/2023'][selected_features_text]
X_val_text = train_nn[train_nn['feature_date'] > '30/08/2023'][selected_features_text]
y_train = train_nn[train_nn['feature_date'] <= '30/08/2023']['target']
y_val = train_nn[train_nn['feature_date'] > '30/08/2023']['target']
w_train = train_nn[train_nn['feature_date'] <= '30/08/2023']['w']
w_val = train_nn[train_nn['feature_date'] > '30/08/2023']['w']
X_train.shape, X_val.shape

In [ ]:
# числовые фичи приведем к типу float
X_train = X_train.astype(float)
X_val = X_val.astype(float)
X_train.shape, X_val.shape

In [ ]:
# нормируем числовые признаки
scaler = MinMaxScaler()
X_train[selected_features_num] = scaler.fit_transform(X_train[selected_features_num])
X_val[selected_features_num] = scaler.transform(X_val[selected_features_num])

In [ ]:
# оставим среди категориальных фичей только признаки без большого количества уникальных значениц
selected_features_cat = ['addrref',
  'oldest_campaignsegment_ccode_for_nss',
  'oldest_campaignsegment_ccode_for_pil',
  'segment',
  'quater']

In [ ]:
# размерность ембеддингов для кат.признаков
embedding_projection = {'addrref': (60, 16),
                        'oldest_campaignsegment_ccode_for_nss': (58, 16),
                        'oldest_campaignsegment_ccode_for_pil': (14, 7),
                        'segment': (4, 3),
                        'quater': (5, 4)
}

In [ ]:
# список значений категориальных фичей для обучения нейросети
list_train_cat = [X_train_cat[feat].values for feat in selected_features_cat]
list_val_cat = [X_val_cat[feat].values for feat in selected_features_cat]

In [ ]:
# Токенизация данных для признака 'main_last_position_ccode_text'
X_train_text = X_train_text['main_last_position_ccode_corr_text']
X_val_text = X_val_text['main_last_position_ccode_corr_text']
tokenizer = Tokenizer()
tokenizer.fit_on_texts(pd.concat([X_train_text, X_val_text, test_df['main_last_position_ccode_corr_text']]))
X_train_seq = tokenizer.texts_to_sequences(X_train_text)
X_val_seq = tokenizer.texts_to_sequences(X_val_text)
n_codes = len(tokenizer.word_index)
print('Количество уникальных слов:', n_codes)

In [ ]:
# обрежем 'main_last_position_ccode_text' до 10 слов, и дозаполним нулями
X_train_pad = pad_sequences(X_train_seq, maxlen=10)
X_val_pad = pad_sequences(X_val_seq, maxlen=10)
X_train_pad.shape, X_val_pad.shape

In [ ]:
# построим модель

inputs = []
cat_embeds = []
embeds_reshape = []

# для каждой кат фичи
for cat_feat in selected_features_cat:
    num_categories = embedding_projection[cat_feat][0]  # количество категорий в признаке
    embedding_size = embedding_projection[cat_feat][1]  # размерность эмбеддингов
    input = Input(shape=(1,))  # входной слой
    embedding = Embedding(num_categories+1, embedding_size, trainable=True)(input) # ембеддинг слой
    emb_reshape = Reshape((embedding_size, ))(embedding)
    inputs.append(input)
    cat_embeds.append(input)
    embeds_reshape.append(emb_reshape)

# text feature
inp = Input(shape=(None, ))  # входной слой
inputs.append(inp)
emb = Embedding(n_codes+1, 64, trainable=True, mask_zero=True)(inp) # ембеддиг слой
dropout_embeds = SpatialDropout1D(0.5)(emb)
sequences = LSTM(10, return_sequences=True)(dropout_embeds)
last_state = LSTM(10)(sequences)

# входной слой для числовых фичей
numeric_input = Input(shape=(len(selected_features_num),))

# конкатенируем эмбеддинги с другими признаками
concatenated = Concatenate()([*embeds_reshape, numeric_input, last_state])

# Добавляем скрытые слои и выходной слой
dense1 = Dense(512, activation='relu')(concatenated)
dense2 = Dense(256, activation='relu')(dense1)
dense3 = Dense(128, activation='relu')(dense2)
dense4 = Dense(64, activation='relu')(dense3)
dense5 = Dense(32, activation='relu')(dense4)
dense6 = Dense(16, activation='relu')(dense5)
output = Dense(1)(dense6)

# Создаем модель
model = Model(inputs=[*inputs, numeric_input], outputs=output)

# Компиляция модели с функцией потерь MAE и параметром sample_weight
model.compile(optimizer='adam', loss='mae', weighted_metrics=['mae'])

# Обучение модели с параметром sample_weight
history = model.fit(
    [*list_train_cat, X_train_pad, X_train], y_train,
    epochs=1, batch_size=128,
    validation_data=([*list_val_cat, X_val_pad, X_val], y_val, w_val),
    sample_weight=w_train
    )

In [ ]:
# указываем путь для сохранения лучшей модели
model_save_path = './drive/MyDrive/SF_DS/data_income/model_nn_best.h5'
checkpoint_callback = ModelCheckpoint(model_save_path,
                                      monitor='val_loss',
                                      save_best_only=True,
                                      verbose=1)

In [ ]:
history = model.fit(
    [*list_train_cat, X_train_pad, X_train], y_train,
    epochs=10, batch_size=128,
    validation_data=([*list_val_cat, X_val_pad, X_val], y_val, w_val),
    sample_weight=w_train,
    callbacks=[checkpoint_callback]
    )

In [ ]:
# Загружаем обученную модель
model = load_model('./drive/MyDrive/SF_DS/data_income/model_nn_best.h5')

In [ ]:
y_val_pred_nn = model.predict([*list_val_cat, X_val_pad, X_val])[:,0]

weighted_mean_absolute_error(y_val, y_val_pred_nn, w_val)

## Сравнение моделей

Проведем сравнение качества моделей по метрике WMAE на валидационной выборке.

Catboost RMSE: 29935.85

Catboost RMSE после логарифмирования таргета: 29693.23

Catboost MAE: 29086.53

Catboost RMSE с оптимизированными гиперпараметрами: 29334.95

Классификатор + Catboost MAE и RMSE: 28680.34

Stacking 2 базовые модели: 29464.99

Stacking 5 базовых моделей: 28900.21

NeuralNetwork: 29839.46

Лучше всего себя показал Классификатор + Catboost MAE и RMSE и Stacking 5 базовых моделей, их и будем использовать для финального предсказания на тестовой выборке.

## KFold валидация

Для получения предсказаний для тестовой выборки будем использовать KFold валидацию на всем тренировочном датасете.

In [ ]:
# KFold для модели с метрикой RMSE

kf = KFold(n_splits=5, shuffle=True, random_state=12)
models = []
metrics_kf = []
metrics_train = []

for j, (train_index, test_index) in enumerate(kf.split(
    train_df[selected_features]
    )):
    model = CatBoostRegressor(
        loss_function='RMSE',
        **best_params,
        iterations = 20000,
        random_state = 42,
        verbose=1000,
        early_stopping_rounds=200,
        cat_features=selected_features_cat,
        text_features=selected_features_text,
        task_type='GPU',
        eval_metric='MAE',
        border_count=254
    )

    train_data = Pool(
        train_df[selected_features].iloc[train_index],
        train_df['target'].iloc[train_index],
        cat_features=selected_features_cat,
        text_features = selected_features_text,
        weight = train_df['w'].iloc[train_index]
    )

    eval_data = Pool(
        train_df[selected_features].iloc[test_index],
        train_df['target'].iloc[test_index],
        cat_features=selected_features_cat,
        text_features = selected_features_text,
        weight = train_df['w'].iloc[test_index]
    )

    model.fit(train_data, eval_set=eval_data)

    preds_test = model.predict(train_df[selected_features].iloc[test_index])
    preds_train = model.predict(train_df[selected_features].iloc[train_index])

    metric_train = weighted_mean_absolute_error(
        train_df['target'].iloc[train_index], preds_train, train_df['w'].iloc[train_index]
    )
    metric = weighted_mean_absolute_error(
        train_df['target'].iloc[test_index], preds_test, train_df['w'].iloc[test_index]
    )

    metrics_train.append(metric_train)
    metrics_kf.append(metric)
    models.append(model)

metrics_kf, metrics_train


In [ ]:
np.mean(metrics_kf)

In [ ]:
# KFold для модели с метрикой MAE

kf = KFold(n_splits=5, shuffle=True, random_state=12)
models_mae = []
metrics_kf = []
metrics_train = []

for j, (train_index, test_index) in enumerate(kf.split(
    train_df[selected_features]
    )):
    model = CatBoostRegressor(
        loss_function='MAE',
        iterations = 20000,
        learning_rate = 4500,
        random_state = 42,
        verbose=1000,
        early_stopping_rounds=200,
        cat_features=selected_features_cat,
        text_features=selected_features_text,
        task_type='GPU',
        eval_metric='MAE',
        border_count=254
    )

    train_data = Pool(
        train_df[selected_features].iloc[train_index],
        train_df['target'].iloc[train_index],
        cat_features=selected_features_cat,
        text_features = selected_features_text,
        weight = train_df['w'].iloc[train_index]
    )

    eval_data = Pool(
        train_df[selected_features].iloc[test_index],
        train_df['target'].iloc[test_index],
        cat_features=selected_features_cat,
        text_features = selected_features_text,
        weight = train_df['w'].iloc[test_index]
    )

    model.fit(train_data, eval_set=eval_data)

    preds_test = model.predict(train_df[selected_features].iloc[test_index])
    preds_train = model.predict(train_df[selected_features].iloc[train_index])

    metric_train = weighted_mean_absolute_error(
        train_df['target'].iloc[train_index], preds_train, train_df['w'].iloc[train_index]
    )
    metric = weighted_mean_absolute_error(
        train_df['target'].iloc[test_index], preds_test, train_df['w'].iloc[test_index]
    )

    metrics_train.append(metric_train)
    metrics_kf.append(metric)
    models_mae.append(model)

metrics_kf, metrics_train


In [ ]:
np.mean(metrics_kf)

## Предсказание для test

Построим предсказание для тестовой выборки, усреднив предсказания лучших моделей, которые мы построили:

- предсказание по вероятностям классификатора
- предсказание стекинга

In [ ]:
# предсказание класса для теста
preds_test_proba = np.zeros([test_df.shape[0], len(models_cl[0].classes_)])

for model_cl in models_cl:
    preds_test_proba += model_cl.predict_proba(test_df[selected_features])/len(models_cl)

In [ ]:
# предсказание на кросс-валидации модели MSE
preds_test_mse = np.zeros(test_df.shape[0])

for model in models:
    preds_test_mse += model.predict(test_df[selected_features])/len(models)

In [ ]:
# предсказание на кросс-валидации модели MAE
preds_test_mae = np.zeros(test_df.shape[0])

for model_cv in models_mae:
    preds_test_mae += np.exp(model_cv.predict(test_df[selected_features]))/len(models)

In [ ]:
# предсказание стекинга
pred_test_st = stacking_reg_5.predict(test_df[selected_features])

In [ ]:
test_df['pred_proba'] = preds_test_proba[:,1]
test_df['pred_mse'] = preds_test_mse
test_df['pred_log'] = preds_test_mae
test_df['pred_cl'] = test_df['pred_mse']*test_df['pred_proba'] + \
test_df['pred_mae']*(1-test_df['pred_proba'])
test_df['pred_st'] = pred_test_st
test_df['predict'] = test_df['pred_cl']*0.5 + test_df['pred_st']*0.5

In [ ]:
test_df[['client_id','predict']].set_index('client_id').to_csv("submit.csv", sep=",", decimal=".")

В итоговое решение, которое было загружено на Kaggle, вошли предсказания описанных выше моделей, а также были добавлены еще предсказания моделей других участников команды с учетом описанного вероятностного подхода.

С этим решением команда заняла второе место со следующими результатами:
- Private 27868
- Public 27370